In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
!pip install pandas
!pip install numpy

In [4]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\hyeju\appdata\local\temp\pip-req-build-6o7hc1_r
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\hyeju\AppData\Local\Temp\pip-req-build-6o7hc1_r'


In [8]:
!git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git C:\Users\hyeju\AppData\Local\Temp\pip-req-build-6o7hc1_r

In [37]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP model and preprocessing
model, preprocess = clip.load('ViT-B/32', device)

# Dataset class definition
class FabricDataset(Dataset):
    def __init__(self, images, labels, ratios):
        self.images = images
        self.labels = labels
        self.ratios = ratios
        self.bilinear = nn.Bilinear(512, len(ratios[0]), 512)  # Bilinear layer for combining image and ratio vectors

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = preprocess(self.images[idx]).to(device)
        with torch.no_grad():
            img_features = model.encode_image(img.unsqueeze(0)).float()
        img_features /= img_features.norm(dim=-1, keepdim=True)

        ratios_tensor = torch.tensor(self.ratios[idx], dtype=torch.float32).to(device)
        combined_features = self.bilinear(img_features, ratios_tensor.unsqueeze(0))

        return combined_features.squeeze(0), self.labels[idx]

# Function to load images
def load_image(image_path):
    return Image.open(image_path).convert('RGB')

# Function to create datasets and dataloaders
def create_dataset_and_dataloader(images, labels, ratios, batch_size=4):
    dataset = FabricDataset(images, labels, ratios)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataset, dataloader

# Function to save embeddings and labels
def save_embeddings_and_labels(dataset, filename):
    embeddings = []
    labels = []
    for features, label in DataLoader(dataset, batch_size=1):
        embeddings.append(features.squeeze(0).detach().cpu().numpy())
        labels.append(label.item())
    embeddings = np.array(embeddings)
    labels = np.array(labels)
    np.savez(filename, embeddings=embeddings, labels=labels)

# Load the CSV file
data = pd.read_csv('C:/Users/hyeju/Downloads/one_multi/hahaha/CLIP.csv')

# Absolute path to the directory containing the images
image_dir = r'C:/Users/hyeju/Downloads/one_multi/hahaha/GGMU'
data['path'] = data['path'].apply(lambda x: os.path.join(image_dir, os.path.basename(x)))

# Load and preprocess all images
images = [load_image(img_path) for img_path in data['path']]

# Prepare ratios by including all 13 material columns
material_columns = ['cotton', 'polyester', 'acril', 'nylon', 'rayon', 'span', 'linen', 'polyurethane', 'modal', 'wool', 'tencel', 'acetate']
ratios = data[material_columns].values.tolist()

# Extract the target properties and convert them to integer labels
properties = ['softness', 'lubricity', 'thickness', 'elasticity']
labels_dict = {prop: data[prop].values for prop in properties}

# Create datasets, dataloaders and save embeddings for each property
for prop in properties:
    dataset, dataloader = create_dataset_and_dataloader(images, labels_dict[prop], ratios)
    save_embeddings_and_labels(dataset, f'C:/Users/hyeju/Downloads/one_multi/{prop}_embeddings_org_bi.npz')

print("Embeddings and labels saved.")

Embeddings and labels saved.


In [38]:
# Function to load embeddings and labels from npz file
def load_embeddings_and_labels(filename):
    data = np.load(filename)
    embeddings = data['embeddings']
    labels = data['labels']
    return embeddings, labels

# Load and print the dimensions of embeddings and labels for each npz file
softness_embeddings, softness_labels = load_embeddings_and_labels('C:/Users/hyeju/Downloads/one_multi/softness_embeddings_org_bi.npz')

print("Softness embeddings shape:", softness_embeddings.shape)
print("Softness labels shape:", softness_labels.shape)


Softness embeddings shape: (86, 512)
Softness labels shape: (86,)


In [39]:
# .npz 파일 로드
npz_file = np.load('C:/Users/hyeju/Downloads/one_multi/lubricity_embeddings_org_add.npz')

# 파일에 저장된 배열의 키 목록 출력
keys = list(npz_file.keys())
print("Keys in the .npz file:", keys)

# 각 키에 대한 배열 출력 (생략 없이)
np.set_printoptions(threshold=np.inf)
for key in keys:
    print(f"Array for key '{key}':")
    print(npz_file[key])
    print()


Keys in the .npz file: ['embeddings', 'labels']
Array for key 'embeddings':
[[ 2.68363744e-01 -2.70379782e-01 -8.19574594e-02  1.32639572e-01
   3.44083756e-01 -2.77112395e-01 -2.97910333e-01  1.66127458e-01
  -1.20168597e-01 -2.09364206e-01 -1.76329538e-01  2.53302366e-01
  -3.14909458e-01 -4.37627770e-02  4.59837615e-02  1.40637290e-02
   2.45850123e-02 -2.28201412e-02 -4.23432350e-01  2.38873847e-02
   5.92518738e-03  6.92782700e-02 -9.13508013e-02 -3.18371713e-01
   6.56215250e-02 -6.46619499e-02 -1.77881151e-01  1.19855888e-01
  -2.62781054e-01  1.04443818e-01  1.22461259e-01  4.74156924e-02
  -6.35682046e-02 -3.31420064e-01  8.15770403e-02 -3.78295273e-01
  -9.26953778e-02 -1.20026983e-01 -3.77181359e-02 -6.44419193e-02
   1.78188041e-01  7.83793405e-02 -1.30674019e-01 -4.96123955e-02
   1.61301136e-01  1.14243627e-01 -1.29688665e-01 -3.91380519e-01
  -1.32956237e-01  9.48338732e-02  1.62519202e-01 -3.20787817e-01
  -2.35961303e-01  3.27247828e-02  1.84467584e-01  1.38682589e-01
